In [1]:
#for Bing
import os
from langchain.utilities import BingSearchAPIWrapper, SerpAPIWrapper

# for Agent
from langchain.agents import AgentExecutor, BaseSingleActionAgent, Tool, ZeroShotAgent
from langchain.schema import AgentAction, AgentFinish

# for LLM
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Maintenance
from typing import Any, List, Tuple, Union

# for Chain
from langchain.chains import LLMChain

In [2]:
SERP_API_KEY = "d38482db0efe9c48ec77ffdd165c3fb4d7144a6bf86b3604e4367b0a0fe17fdd"
google_params = {
    "engine": "google",
    "hl": "en"
}
google_API_wrapper = SerpAPIWrapper(serpapi_api_key=SERP_API_KEY, params=google_params)
google_tool =  Tool(
    name="Google",
    func=google_API_wrapper.run,
    description="Useful for when you need short answers (e.g., a single number) to questions about current state of affairs",
    return_direct=True)

os.environ["BING_SUBSCRIPTION_KEY"] = "02fa2d164e534dd69f2a586caba57cb6"
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

bing_API_wrapper = BingSearchAPIWrapper()
bing_tool =  Tool(
    name="Bing",
    func=bing_API_wrapper.run,
    description="useful for when you need descriptive answers questions about current state of affairs",
    return_direct=True)

tools = [bing_tool, google_tool]

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q6_K.gguf",
    temperature=0.1,
    max_tokens=500000,
    top_p=0.9,
    callback_manager=callback_manager,
    verbose=True,
)

# Define the prompt for the LLMChain
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix="Determine if the following question can be answered with your current knowledge. If not, you have access to following online search tools. You may try to break down the question to steps.",
    suffix="'Begin!\n\nQuestion: {input}\nThought:{agent_scratchpad}',",
    input_variables=["input", "agent_scratchpad"]
)

# Create the LLMChain with OpenAI model
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define the agent and agent executor
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

# Example question
question = "Who is a British king now??"

# Run the agent
response = agent_executor.run(question)
print(response)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 




> Entering new AgentExecutor chain...
 I should check if there is any current information about British kings.
Action: Google (input: 'British king')
Action Input: https://www.google.com/search?q=British+king I should check if there is any current information about British kings.
Action: Google (input: 'British king')
Action Input: https://www.google.com/search?q=British+king
Observation: Google (input: 'British king') is not a valid tool, try one of [Bing, Google].
Thought:

Llama.generate: prefix-match hit


 I will check Bing instead.
Action: Bing (input: 'British king')
Action Input: https://www.bing.com/search?q=British+king I will check Bing instead.
Action: Bing (input: 'British king')
Action Input: https://www.bing.com/search?q=British+king
Observation: Bing (input: 'British king') is not a valid tool, try one of [Bing, Google].
Thought:

Llama.generate: prefix-match hit


KeyboardInterrupt: 

In [9]:
agent.__dict__

{'llm_chain': LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template="Determine if the following question can be answered with your current knowledge. If not, you have access to following online search tools. You may try to break down the question to steps.\n\nBing: useful for when you need descriptive answers questions about current state of affairs\nGoogle: Useful for when you need short answers (e.g., a single number) to questions about current state of affairs\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Bing, Google]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n'Begin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [35]:
class FakeAgent(BaseSingleActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="Google", tool_input=kwargs["input"], log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="Google", tool_input=kwargs["input"], log="")

In [36]:
agent = FakeAgent()

In [37]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [38]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
38,781,291 people


> Finished chain.


'38,781,291 people'